# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
# Csv file 
weather_df_csv = '../Output/weather_df.csv'
weather_df = pd.read_csv(weather_df_csv)
dropna = weather_df.dropna()
del dropna["Unnamed: 0"]
dropna.head(10)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,luoyang,34.68,112.45,60.01,99,56,7.78,CN,1603226235
1,arraial do cabo,-22.97,-42.03,77.00,73,0,18.34,BR,1603225956
2,new norfolk,-42.78,147.06,49.28,71,99,1.92,AU,1603225941
3,tasiilaq,65.61,-37.64,32.00,93,100,21.92,GL,1603226235
4,hithadhoo,-0.60,73.08,82.15,80,97,14.03,MV,1603226236
5,ondjiva,-17.07,15.73,76.42,17,0,0.56,AO,1603226236
6,axim,4.87,-2.24,77.29,88,99,9.01,GH,1603226236
7,saskylakh,71.92,114.08,20.84,97,99,7.76,RU,1603225956
8,rikitea,-23.12,-134.97,73.51,79,0,14.97,PF,1603225976
9,carnarvon,-24.87,113.63,66.20,82,38,14.99,AU,1603225957


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# locations
loc = dropna[['Lat', 'Lng']]
humidity = dropna['Humidity'].astype(float)

In [18]:
# Plot Heatmap
Heatmap = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(loc, weights=humidity, 
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=4)

# Heatmap layer
Heatmap.add_layer(heat_layer)
Heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
max_temp = dropna[dropna['Max Temp'] < 80]
min_temp = max_temp[max_temp['Max Temp'] > 70]
cloudiness = min_temp[min_temp['Cloudiness'] == 0]
ideal_conditions = cloudiness[cloudiness['Wind Speed'] < 20]
ideal_conditions = ideal_conditions.dropna()
indexed_ideal_conditions = ideal_conditions.reset_index()
indexed_ideal_conditions

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,1,arraial do cabo,-22.97,-42.03,77.00,73,0,18.34,BR,1603225956
1,5,ondjiva,-17.07,15.73,76.42,17,0,0.56,AO,1603226236
2,8,rikitea,-23.12,-134.97,73.51,79,0,14.97,PF,1603225976
3,126,angoche,-16.23,39.91,76.48,80,0,7.65,MZ,1603226263
4,146,poum,-20.23,164.02,74.19,69,0,5.37,NC,1603226267
5,158,san quintin,30.48,-115.95,73.76,68,0,16.20,MX,1603226025
6,185,tura,25.52,90.22,72.07,92,0,2.89,IN,1603226275
7,202,inhambane,-23.86,35.38,73.49,90,0,9.01,MZ,1603226279
8,255,maxixe,-23.86,35.35,72.82,91,0,8.19,MZ,1603226291
9,276,sardarshahr,28.43,74.48,78.44,17,0,8.63,IN,1603226295


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
info = pd.DataFrame({'City':indexed_ideal_conditions['City'], 
                           'Country':indexed_ideal_conditions['Country'], 
                           'Lat': indexed_ideal_conditions['Lat'],
                           'Lng': indexed_ideal_conditions['Lng']})
info.head()
    

,City,Country,Lat,Lng
0,arraial do cabo,BR,-22.97,-42.03
1,ondjiva,AO,-17.07,15.73
2,rikitea,PF,-23.12,-134.97
3,angoche,MZ,-16.23,39.91
4,poum,NC,-20.23,164.02


In [31]:
# Hotel variable
hotels = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'  
params = { 'types': 'lodging', 'radius': 50000000, 'key': g_key}
# Loop for closest hotel
for city in range(len(indexed_ideal_conditions['City'])):
    lat = indexed_ideal_conditions.loc[city]['Lat']
    lng = indexed_ideal_conditions.loc[city]['Lng']

    city_coords = f'{lat},{lng}'
    params['location'] = city_coords
    
    request = requests.get(base_url, params=params)
    response = request.json()

    try:
        hotels.append(response['results'][0]['name'])
    except:
        hotels.append('No close hotel found')

# Nearest hotel
indexed_ideal_conditions['Hotel Name'] = hotels
hotel_df = indexed_ideal_conditions
hotel_df

#I can't get any hotel names to show up. I have tried many different things and cant get this to work.

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,1,arraial do cabo,-22.97,-42.03,77.00,73,0,18.34,BR,1603225956,No close hotel found
1,5,ondjiva,-17.07,15.73,76.42,17,0,0.56,AO,1603226236,No close hotel found
2,8,rikitea,-23.12,-134.97,73.51,79,0,14.97,PF,1603225976,No close hotel found
3,126,angoche,-16.23,39.91,76.48,80,0,7.65,MZ,1603226263,No close hotel found
4,146,poum,-20.23,164.02,74.19,69,0,5.37,NC,1603226267,No close hotel found
5,158,san quintin,30.48,-115.95,73.76,68,0,16.20,MX,1603226025,No close hotel found
6,185,tura,25.52,90.22,72.07,92,0,2.89,IN,1603226275,No close hotel found
7,202,inhambane,-23.86,35.38,73.49,90,0,9.01,MZ,1603226279,No close hotel found
8,255,maxixe,-23.86,35.35,72.82,91,0,8.19,MZ,1603226291,No close hotel found
9,276,sardarshahr,28.43,74.48,78.44,17,0,8.63,IN,1603226295,No close hotel found


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_df = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [32]:
# I belive this code is correct 
marker = gmaps.marker(loc, info_box_content=hotel_df)

# Add marker layer ontop of heat map
fig.add_layer(marker)
# Display figure
fig

TypeError: 'module' object is not callable